In [0]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

Given a set of $n + 1$ data points ($x_i,y_i$) where $x_0<x_1<\cdots<x_n$

The spline S(x) is a function satisfying, that on each subinterval $[x_{i-1},x_{i}], S(x)$ is a polynomial of degree 3, where $i=1,\cdots,n.$
And $S(x_i)=y_i,$ for all $i=0,1,\cdots,n.$ Formula:

$S(x)=\left\{\begin{array}{rl}
P_1(x), & x_0 \leq x \leq x_1\\
\cdots &\\
P_n(x), & x_{n-1} < x \leq x_n\end{array}\right.$

where each $P_i=a_{0, i}+a_{1, i}x+a_{2, i}x^2+a_{3, i}x^3 $ is a cubic function (polynomial), $i=1,\cdots,n$.

So, problem is to find polynomials that will have 2nd derivatives in union:

$S_3(x) = \bigcup_{i=0}^{n-1}S_{3,i}(x) \in C_2[a, b]$, where $a = x_0$ and $b = x_n$

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

Functional condition:

$\delta S_{3, i}(x)|_{x = x_{i}} = S_{3, i}(x)|_{x = x_{i}} - f(x_{i}) = 0$, where $i \in [0..n]$.

Differential conditions:

$\delta S_{3, i}^{'}(x)|_{x = x_{i}} = S_{3, i}^{'}(x)|_{x = x_{i}} - f^{'}(x_{i}) = 0$, where $i \in [0..n]$.

$\delta S_{3, i}^{''}(x)|_{x = x_{i}} = S_{3, i}^{''}(x)|_{x = x_{i}} - f^{''}(x_{i}) = 0$, where $i \in [0..n]$.










### 3. Formulate stitching conditions [mathematical formula]

$S^{(p)}_{3, i}(x)|_{x = x_{i}} = S^{(p)}_{3, i}(x)|_{x = x_{i}}$, where $i \in [1..n - 1]$.

### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

Stiching conditions along with differential matching conditions ensure regularity $S^{p}_3(x), p=0,1,2$ in all internal dots $x_i$, i.e. $(x_i, S_3(x_i)), (i=\overline{1,n-1})$. 

This ensures that the conditions $S_m(x) \in C_2[a,b]$ are met.

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

From the splain equation we have system of linear equations:

$\left\{\begin{array}\\
S_{3, i}(x)|_{x=x_{i}} = f(x_{i})\\
S_{3, i}(x)|_{x=x_{i+1}} = f(x_{i+1})\\
S^{(2)}_{3, i}(x)|_{x=x_{i}} = f^{(2)}(x_{i})\\
S^{(2)}_{3, i}(x)|_{x=x_{i+1}} = f^{(2)}(x_{i+1})\end{array}\right.$

From this we have:

$\left\{\begin{array}{rl}
a_{0, i} - f_i(x_i) = 0\\
\sum^3_{j = 0}a_{j, i}(x_{i + 1} - x_i)^j - f(x_{x + 1}) = 0 \\
2a_{2, i} - m_i = 0\\
2a_{2, i}+6a_{3, i}(x_{i + 1} - x_i) - m_{i + 1} = 0\end{array}\right.$,

where $m_{i = 1}$


To find parameters we can have following matrix:

$
\begin{bmatrix} 
1 & 0 & 0 & 0 \\
1 & h_{i + 1} & h_{i + 1}^2 & h_{i + 1}^3\\
0 & 0 & 2 & 0 \\
1 & 0 & 2 & 6h_{i + 1} \\
\end{bmatrix}
\quad
\begin{bmatrix} 
a_{0, i}\\
a_{1, i}\\
a_{2, i}\\
a_{3, i}\\
\end{bmatrix}
\quad
=
\quad
\begin{bmatrix} 
f_{i}\\
f_{i + 1}\\
m_{i}\\
m_{i + 1}\\
\end{bmatrix}
$

where $h_{i + 1} = (x_{i + 1} - x_i)$

From this matrix we can compute parameters $a_{j, i}$. Then we use them in initial spline equation:

$S_{3, i} = f_{i} + (\frac{\Delta f_{i}}{h_{i+1}} - \frac{h_{i+1}}{2}m_{i} - \frac{h_{i+1}}{6}\Delta m_{i})(x-x_{i}) + \frac{m_{i}}{2}(x-x_{i})^{2} + \frac{1}{6h_{i+1}}\Delta m_{i}(x-x_{i})^{3}$, $i \in [0..n-1]$

Then we use stiching condition:

$S^{'}_{3, i - 1}(x)|_{x = x_{i}} = S^{'}_{3, i}(x)|_{x = x_{i}}$

After difference of these equations with observed pararmeters we get:

$\frac{\Delta f_{i-1}}{h_{i}} - \frac{h_{i}m_{i-1}}{2}-\frac{h_{i}\Delta m_{i-1}}{6} + m_{i-1}h_{i} + \frac{\Delta m_{i-1}}{2h_{i}}h_{i}^{2} = \frac{\Delta f_{i}}{h_{i+1}} - \frac{h_{i+1}m_{i}}{2} - \frac{h_{i+1}\Delta m_{i}}{6}$

$\frac{h_{i}}{6}m_{i-1} + \frac{h_{i} + h_{i+1}}{3}m_{i} + \frac{h_{i+1}}{6}m_{i+1} = \frac{\Delta f_{i}}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}}$

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

Using above formula we have following matrices:

$\begin{bmatrix}
\frac{h_{1}}{6} & \frac{h_{1} + h_{2}}{3} & \frac{h_{2}}{6} &  0 & ... & ...\\
0 & \frac{h_{2}}{6}& \frac{h_{2} + h_{3}}{3} & \frac{h_{3}}{6} & 0 & ...\\
... & ... & ... & ... & ... & ...\\
... & 0 & \frac{h_{n-2}}{6} & \frac{h_{n-2} + h_{n-1}}{3} & \frac{h_{n-1}}{6} & 0\\
... & ...  & 0 & \frac{h_{n-1}}{6} & \frac{h_{n-1} + h_{n}}{3} & \frac{h_{n}}{6}\\
\end{bmatrix}
\quad
\begin{bmatrix}
m_{0}\\
m_{1}\\
...\\
m_{n-1}\\
m_{n} \\
\end{bmatrix}
\quad
=
\quad
\begin{bmatrix}
\frac{\Delta f_{1}}{h_{2}} - \frac{\Delta f_{0}}{h_{1}}\\
\frac{\Delta f_{2}}{h_{3}} - \frac{\Delta f_{1}}{h_{2}}\\
...\\
\frac{\Delta f_{n-2}}{h_{n-1}} - \frac{\Delta f_{n-3}}{h_{n-2}}\\
\frac{\Delta f_{n-1}}{h_{n}} - \frac{\Delta f_{n-2}}{h_{n-1}}
\end{bmatrix}$

### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

Uknowns are second derivatives of the function.

System is open, as we have only $n - 1$ equations (dots of stiching), but also have $n + 1$ unknowns (all dots), so we have not 2 equations for closure.

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

Firstly, we obtain system closure with assuming $m_0=m_n=0$

Then we apply Gaussian elimination (common approach):

$\begin{bmatrix}
-b_{1} & c_{1} &  0 & ... & ...\\ 
a_{1} & -b_{2} & c_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{bmatrix} 
\quad
\bar{x} 
\quad
=
\quad 
\begin{bmatrix}
d_{1}\\ 
d_{2}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{bmatrix}$

Divide first row by $-b_{1}$:

$\begin{bmatrix}
1 & -\frac{c_{1}}{b_{1}} &  0 & ... & ...\\ 
a_{1} & -b_{2} & c_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{bmatrix} 
\quad
\bar{x}
\quad
=
\quad
\begin{bmatrix}
-\frac{d_{1}}{b_{1}}\\ 
d_{2}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{bmatrix}$

Set $P_{1} = \frac{c_{1}}{b_{1}}$, $Q_{1} = -\frac{d_{1}}{b_{1}}$. Then $x_{1} = P_{1}x_{2} + Q_{1}$.

$\begin{bmatrix}
1 & -P_{1} &  0 & ... & ...\\ 
a_{1} & -b_{2} & c_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{bmatrix}
\quad
\bar{x}
\quad
=
\quad
\begin{bmatrix}
Q_{1}\\ 
d_{2}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{bmatrix}$

Multiply first row by $a_{1}$ and subtract it from second row:

$\begin{bmatrix}
1 & -P_{1} &  0 & ... & ...\\ 
0 & a_{2}P_{1} -b_{2} & c_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{bmatrix}
\quad
\bar{x}
\quad
=
\quad
\begin{bmatrix}
Q_{1}\\ 
d_{2} - a_{2}Q_{1}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{bmatrix}$

Divide second row by $-b_{2} + a_{2}P_{1}$:

$\begin{bmatrix}
1 & -P_{1} &  0 & ... & ...\\ 
0 & 1 & \frac{c_{2}}{a_{2}P_{1} -b_{2}} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{bmatrix}
\quad
\bar{x}
\quad
=
\quad
\begin{bmatrix}
Q_{1}\\ 
\frac{d_{2} - a_{2}Q_{1}}{a_{2}P_{1} -b_{2}}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{bmatrix}$

Set $P_{2} = -\frac{c_{2}}{a_{2}P_{1} -b_{2}}$, $Q_{2} = \frac{d_{2} - a_{2}Q_{1}}{a_{2}P_{1} -b_{2}}$. Then $x_{2} = P_{2}x_{3} + Q_{2}$.

$\begin{bmatrix}
1 & -P_{1} &  0 & ... & ...\\ 
0 & 1 & -P_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{bmatrix}
\quad
\bar{x}
\quad
=
\quad
\begin{bmatrix}
Q_{1}\\ 
Q_{2}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{bmatrix}$

Steps of Gauss elimination after this one are obvious and recursive derivation of $P$'s and $Q$'s is clear, too.

To be precise, $x_{i-1} = P_{i-1}x_{i} + Q_{i-1}$, where $P_{i-1} = -\frac{c_{i-1}}{a_{i-1}P_{i-2} + b_{i-1}}$, $Q_{i-1} = \frac{d_{i-1} - a_{i-1}Q_{i-2}}{a_{i-1}P_{i-2} - b_{i-1}}$. $P_{1}$ and $Q_{1}$ were defined above.

### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

As we found equation for $x_n$ in the above method (direct pass):
$a_{n}(P_{n-1}x_{n} + Q_{n}) + b_{n}x_{n} = d_{n}$
,
we can find all previous $x_{i}$ by substituting value of $x_{i+1}$ into formula for $x_{i}$ (reverse pass).


### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [2]:
import numpy as np

A = np.array([
    [2.04, -1, 0, 0],
    [-1, 2.04, -1, 0],
    [0, -1, 2.04, -1],
    [0, 0, -1, 2.04]
])
d = np.transpose(np.array([[40.8, 0.8, 0.8, 200.8]]))
n = len(d)


def direct_pass(A, d):
    """
    Performs direct pass of Tridiagonal matrix algorithm
    :param A: tridiagonal matrix
    :param d: vector of the right hand side
    :return: P and Q - using variables
    """
    global n

    P = np.zeros(n - 1)
    Q = np.zeros(n - 1)
    P[0] = A[0, 1] / -A[0, 0]
    Q[0] = d[0] / A[0, 0]
    for i in range(1, n - 1):
        a_i = A[i, i - 1]
        b_i = -A[i, i]
        c_i = A[i, i + 1]
        d_i = d[i]
        P[i] = c_i / (-a_i * P[i - 1] + b_i)
        Q[i] = (-d_i + a_i * Q[i - 1]) / (-a_i * P[i - 1] + b_i)

    return P, Q


def reverse_pass(A, d, P, Q):
    """
    Performs reverse pass of Tridiagonal matrix algorithm
    :param A: tridiagonal matrix
    :param d: vector of the right hand side
    :param P, Q - using variables
    :return: vector of found unknown variables
    """
    global n

    # Compute last row
    res = np.zeros(n)
    res[n - 1] = (d[n - 1] - A[n - 1, n - 2] * Q[n - 2]) / (A[n - 1, n - 2] * P[n - 2] + A[n - 1, n - 1])

    # Perform reverse pass
    for i in range(n - 2, -1, -1):
        res[i] = res[i + 1] * P[i] + Q[i]

    return res


def tridiagonal_matrix_algorithm(A, d):
    """
    Applies Tridiagonal matrix algorithm to Ax = d equation.
    :param A: tridiagonal matrix
    :param d: vector of the right hand side
    :return: vector of found unknown variables
    """
    P, Q = direct_pass(A, d)

    x = reverse_pass(A, d, P, Q)

    return x


print(tridiagonal_matrix_algorithm(A, d))


[ 65.96983437  93.77846211 124.53822833 159.47952369]


### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

It is proved that on a uniform grid a spline function $S_{3, i}$ converges to $f(x) \in C_{4}[a, b]$ with fourth order, and following estimatations are valid:

$||f^{(p)}(x) - S^{(p)} _{3}(x)||_{C[a, b]} = \max _{[a, b]} |f^{(p)}(x) - S^{(p)} _{3}(x)| \leq M_{4}h^{4-p}$, $p = 0, 1, 2.$

Where $M_4 = \max_{[a,b]}|f^{(4)}(x)|$.

On an ununiform grid:

$||f^{(p)}(x) - S^{(p)} _{3}(x)||_{C[a, b]} \leq M_{4}h_{max}^{4-p}$, где $h_{max} = \max _{1 \leq i \leq n}h_{i}$.

### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

Complexity of the algorithm is $O(n)$, as complexity of direct pass is $O(n)$ and complexity of reverse pass is $O(n)$.



### Congrats!